In [40]:
import os
import re
import math
import json
from dotenv import load_dotenv
from groq import Groq

from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.utils.extraction import extract_tag_content

load_dotenv()

MODEL = "llama3-70b-8192"
GROQ_CLIENT = Groq()

## System prompt

In [31]:
REACT_SYSTEM_PROMPT = """
You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
%s
</tools>

Example session:

<question>What's the current temperature in Madrid?</question>
<thought>I need to get the current weather in Madrid</thought>
<tool_call>{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}, "id": 0}</tool_call>

You will be called again with this:

<observation>{0: {"temperature": 25, "unit": "celsius"}}</observation>

You then output:

<response>The current temperature in Madrid is 25 degrees Celsius</response>

Additional constraints:

- If the user asks you something unrelated to any of the tools above, answer freely enclosing your answer with <response></response> tags.
"""

Defining tools

In [32]:
@tool
def sum_two_elements(a:int,b:int)->int:
    """
    Computes the sum of two integers.

    Args:
        a (int): The first integer to be summed.
        b (int): The second integer to be summed.

    Returns:
        int: The sum of `a` and `b`.
    """
    return a + b

@tool
def multiply_two_elements(a:int,b:int)->int:
    """
    Multiplies two integers.

    Args:
        a (int): The first integer to multiply.
        b (int): The second integer to multiply.

    Returns:
        int: The product of `a` and `b`.
    """
    return a * b

@tool
def compute_log(x:int)-> float | str:
    """
    Computes the logarithm of an integer `x` with an optional base.

    Args:
        x (int): The integer value for which the logarithm is computed. Must be greater than 0.

    Returns:
        float: The logarithm of `x` to the specified `base`.
    """
    if x <=0:
        return "Logarthim is undefined for values less than or equal to 0"
    return math.log(x)

available_tools = {
    "sum_two_elements":sum_two_elements,
    "multiply_two_elements":multiply_two_elements,
    "compute_log":compute_log
}    


In [33]:
print("Tool name: ", sum_two_elements.name)
print("Tool signature: ", sum_two_elements.fn_signature)

Tool name:  sum_two_elements
Tool signature:  {"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}


concatinate tool signature with user prompt

In [34]:
tools_signature = sum_two_elements.fn_signature +",/n"+ multiply_two_elements.fn_signature +",/n"+ compute_log.fn_signature

In [35]:
print(tools_signature)

{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},/n{"name": "multiply_two_elements", "description": "\n    Multiplies two integers.\n\n    Args:\n        a (int): The first integer to multiply.\n        b (int): The second integer to multiply.\n\n    Returns:\n        int: The product of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},/n{"name": "compute_log", "description": "\n    Computes the logarithm of an integer `x` with an optional base.\n\n    Args:\n        x (int): The integer value for which the logarithm is computed. Must be greater than 0.\n\n    Returns:\n        float: The logarithm of `x` to the specified `base`.\n    ", "parameters": {"pr

In [36]:
REACT_SYSTEM_PROMPT = REACT_SYSTEM_PROMPT % tools_signature

In [37]:
print(REACT_SYSTEM_PROMPT)


You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"pr

ReACT LOOP 1

In [80]:
USER_QUESTION = "I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result"
chat_history = [
    {
        "role": "system",
        "content": REACT_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": f"<question>{USER_QUESTION}</question>"
    }
]

In [81]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I need to follow the user's instructions step by step</thought>
<tool_call>{"name": "sum_two_elements","arguments": {"a": 1234, "b": 5678}, "id": 0}</tool_call>


In [82]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

ReACT LOOP 2

In [83]:
tool_call = extract_tag_content(output, tag="tool_call")

In [84]:
tool_call

TagContentResult(content=['{"name": "sum_two_elements","arguments": {"a": 1234, "b": 5678}, "id": 0}'], found=True)

In [85]:
tool_call = json.loads(tool_call.content[0])

In [86]:
tool_call

{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 0}

In [87]:
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [88]:
assert tool_result == 1234 + 5678

In [89]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

ReACT LOOP 3

In [90]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I have the sum, now I need to multiply it by 5</thought>
<tool_call>{"name": "multiply_two_elements","arguments": {"a": 6912, "b": 5}, "id": 1}</tool_call>


In [91]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

ReACT LOOP 4 

In [92]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [93]:
print(tool_result)

34560


In [94]:
assert tool_result == (1234 + 5678) * 5

In [95]:
chat_history.append(
    {
        "role":"user",
        "content":f"<observation>{tool_result}</observation>"
    }
)

ReACT LOOP 5

In [96]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I have the result of the multiplication, now I need to compute its logarithm</thought>
<tool_call>{"name": "compute_log","arguments": {"x": 34560}, "id": 2}</tool_call>


In [97]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

ReACT LOOP 6

In [98]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [99]:
tool_result

10.450452222917992

In [100]:
assert tool_result == math.log((1234 + 5678) * 5)

In [101]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

ReACT LOOP 7

In [102]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<response>The result of the calculation is 10.450452222917992</response>


## Using agentic_pattern library

In [104]:
from agentic_patterns.planning_pattern.react_agent import ReactAgent

agent = ReactAgent(tools=[sum_two_elements,multiply_two_elements,compute_log],model=MODEL)

agent.run(user_msg="I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result")


Thought: I need to perform a series of operations: sum of two numbers, multiplication, and logarithm computation.

Using Tool: sum_two_elements

Tool call dict: 
{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 0}

Tool result: 
6912

Observations: {0: 6912}

Thought: I have the sum, now I need to multiply it by 5.

Using Tool: multiply_two_elements

Tool call dict: 
{'name': 'multiply_two_elements', 'arguments': {'a': 6912, 'b': 5}, 'id': 1}

Tool result: 
34560

Observations: {1: 34560}

Thought: I have the result of the multiplication, now I need to compute the logarithm.

Using Tool: compute_log

Tool call dict: 
{'name': 'compute_log', 'arguments': {'x': 34560}, 'id': 2}

Tool result: 
10.450452222917992

Observations: {2: 10.450452222917992}


'The result of the operations is 10.450452222917992.'